In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
dataset_train = pd.read_csv('Salestrain.csv')

In [ ]:
dataset_train.shape

In [ ]:
dataset_train

In [ ]:
plt.plot(dataset_train, color="blue", label="Vendas")
plt.title("Vendas")
plt.xlabel("Tempo")
plt.ylabel("Vendas")
plt.legend()
plt.show()

In [ ]:
# Dados na mesma escala (entre 0 e 1)
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(dataset_train)

In [ ]:
# Formatar os dados para que  cada amostra de entrada (X_train) contenha 90 pontos de tempo consecutivos, e a saída (y_train) seja o próximo valor da série
X_train = []
y_train = []
for i in range(90, len(training_set_scaled)):
  data = training_set_scaled[i-90:i,0] # Seleciona uma janela deslizante de 90 valores anteriores como entrada
  X_train.append(data)
  y_train.append(training_set_scaled[i,0]) # O próximo valor na sequência é armazenado como o rótulo de saída esperado

X_train = np.array(X_train).reshape(-1,90,1)
y_train = np.array(y_train)

# janela deslizante -> lidam com previsão de valores futuros baseados no histórico

In [ ]:
# Criando a rede
modelo = Sequential()
modelo.add(LSTM(units=100, return_sequences=True, input_shape=(X_train.shape[1],1)))
modelo.add(Dropout(0.2)) # Exlui neuronios de forma aleatória, para evitar overfiting
modelo.add(LSTM(units=100, return_sequences=True))
modelo.add(Dropout(0.2))
modelo.add(LSTM(units=100, return_sequences=True))
modelo.add(Dropout(0.2))
modelo.add(LSTM(units=100)) # padrão do return_sequences é false
modelo.add(Dropout(0.2))
modelo.add(Dense(units=1)) # Saída

In [ ]:
# Compilando e treinando o modelo
modelo.compile(optimizer='adam', loss='mean_squared_error')
modelo.fit(X_train, y_train, epochs=300, batch_size=1)

In [ ]:
dataset_test = pd.read_csv('Salestest.csv')

In [ ]:
dataset_test.shape

In [ ]:
# Criando um gráfico com ambos os dados, de teste e treino
train_values = dataset_train['data'].values
test_values = dataset_test['data'].values
total_values = np.concatenate((train_values, test_values), axis=0)

time_index = range(len(total_values))
# Até o comprimento de train_values
plt.plot(time_index[:len(train_values)], train_values, color='blue', label='Vendas - Treinamento',)
# A partir do comprimento de train_values
plt.plot(time_index[len(train_values):], test_values, color='red', label='Vendas - Teste')
plt.title('Vendas')
plt.xlabel('Tempo')
plt.ylabel('Vendas')
plt.legend()
plt.show()

In [ ]:
# Cópia dos dados de teste
dataset_test_anomalies = dataset_test.copy()
dataset_test_anomalies.loc[:9, 'data'] = 90
dataset_test_anomalies.loc[10:34, 'data'] = np.random.uniform(100,200, size=(25,))
dataset_test_anomalies.loc[35:, 'data'] = 90

In [ ]:
# Gráfico para comparar os dados de teste com e sem anomalias
plt.plot(dataset_test, color='blue', label='Vendas')
plt.plot(dataset_test_anomalies, color='red', label='Vendas com Anomalias')
plt.title('Vendas')
plt.xlabel('Tempo')
plt.ylabel('Vendas')
plt.legend()
plt.show()

In [ ]:
dataset_total = pd.concat((dataset_train['data'], dataset_test['data']), axis=0) # Concatenar os dados de treino e teste
inputs = dataset_total[len(dataset_total) - len(dataset_test) -90:] # Cria inputs que serão usados para prever os valores (pega os 90 valores anteriores ao
                                                                    # início do conjunto de teste + Todos os valores do conjunto de teste) -> janela de 90 períodos
inputs = pd.DataFrame(inputs, columns=['data']) # Converte os dados selecionados para um DataFrame Pandas com uma única coluna chamada 'data'
inputs = sc.transform(inputs)

In [ ]:
dataset_total_anomalies = pd.concat((dataset_train['data'], dataset_test_anomalies['data']), axis=0)
inputs_anomalies = dataset_total_anomalies[len(dataset_total_anomalies) - len(dataset_test_anomalies) -90:]
inputs_anomalies = pd.DataFrame(inputs_anomalies, columns=['data'])
inputs_anomalies = sc.transform(inputs_anomalies)

In [ ]:
# Criar a janela deslizante com as 90 observações, que nem foi feito com os dados de treino
X_test = []
X_test_anomalies = []

for i in range (90, len(inputs)):
  X_test.append(inputs[i-90:i,0])
  X_test_anomalies.append(inputs_anomalies[i-90:i,0])

X_test, X_test_anomalies = np.array(X_test), np.array(X_test_anomalies)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_test_anomalies = np.reshape(X_test_anomalies, (X_test_anomalies.shape[0], X_test_anomalies.shape[1], 1))

In [ ]:
# Fazendo as previsões
predicted_sales = modelo.predict(X_test)
predicted_sales = sc.inverse_transform(predicted_sales)

predicted_sales_anomalies = modelo.predict(X_test_anomalies)
predicted_sales_anomalies = sc.inverse_transform(predicted_sales_anomalies)

mes_test = mean_squared_error(test_values, predicted_sales)
mes_test_anomalies = mean_squared_error(test_values, predicted_sales_anomalies)

In [ ]:
print(f'MSE para conjunto de dados normal: {mes_test}')
print(f'MSE para conjunto de dados com anomalias: {mes_test_anomalies}')

In [ ]:
plt.plot(test_values, color='blue', label='Valores Reais')
plt.plot(predicted_sales, color='red', label='Previsões')
plt.plot(predicted_sales_anomalies, color='green', label='Previsões com Anomalias')
plt.xlabel('Tempo')
plt.ylabel('Vendas')
plt.legend()
plt.show()